In [1]:
import copy
import math
import numpy as np
import sys
import vtk
from numpy import linalg as LA

In [8]:
filename = '../../data/Isabel_pressure_velocity_qvapor.vti'
reader = vtk.vtkXMLImageDataReader()
reader.SetFileName(filename)
reader.Update()
data = reader.GetOutput()
dims = data.GetDimensions()
var1='Pressure'
var2='Velocity'

In [9]:
tevfield = vtk.vtkImageData()
tevfield.SetDimensions(data.GetDimensions())
tevfield.GetPointData().AddArray(data.GetPointData().GetArray(var1))

arr = np.ndarray(shape=(dims[0],dims[1],dims[2]), dtype=float)
index=0
for i in range(dims[2]):
    for j in range(dims[1]):
        for k in range(dims[0]):
            arr[k,j,i] = data.GetPointData().GetArray(var1).GetTuple1(index)
            index=index+1

gradient_tev = np.gradient(arr)   

# compute magniture
# xx = gradient_tev[0][:][:][:]*gradient_tev[0][:][:][:] + gradient_tev[1][:][:][:]*gradient_tev[1][:][:][:] \
# + gradient_tev[2][:][:][:]*gradient_tev[2][:][:][:]
# grad_mag = np.sqrt(xx)
# print np.min(grad_mag), np.max(grad_mag)

# grad_mag_arr = vtk.vtkDoubleArray()
# grad_mag_arr.SetName('grad_mag')
# grad_mag_arr.SetNumberOfTuples(dims[0]*dims[1]*dims[2])

# index=0
# for i in range(dims[2]):
#     for j in range(dims[1]):
#         for k in range(dims[0]):
#             grad_mag_arr.SetTuple1(index,grad_mag[k,j,i])
#             index=index+1
            
#tevfield.GetPointData().AddArray(grad_mag_arr)   

In [10]:
vo2field = vtk.vtkImageData()
vo2field.SetDimensions(data.GetDimensions())
vo2field.GetPointData().AddArray(data.GetPointData().GetArray(var2))

arr1 = np.ndarray(shape=(dims[0],dims[1],dims[2]), dtype=float)
index=0
for i in range(dims[2]):
    for j in range(dims[1]):
        for k in range(dims[0]):
            arr1[k,j,i] = data.GetPointData().GetArray(var2).GetTuple1(index)
            index=index+1

gradient_vo2 = np.gradient(arr1) 

# # compute magniture
# yy = gradient_vo2[0][:][:][:]*gradient_vo2[0][:][:][:] + gradient_vo2[1][:][:][:]*gradient_vo2[1][:][:][:] \
# + gradient_vo2[2][:][:][:]*gradient_vo2[2][:][:][:]
# grad_mag_1 = np.sqrt(yy)
# print np.min(grad_mag_1), np.max(grad_mag_1)

In [11]:
print np.shape(gradient_vo2), np.shape(gradient_tev)
zz = np.ndarray(shape=(dims[0],dims[1],dims[2]), dtype=float)

for i in range(dims[2]):
    for j in range(dims[1]):
        for k in range(dims[0]):
            v1 = [gradient_tev[0][k][j][i], gradient_tev[1][k][j][i], gradient_tev[2][k][j][i]]
            v2 = [gradient_vo2[0][k][j][i], gradient_vo2[1][k][j][i], gradient_vo2[2][k][j][i]]
            
            v1 = np.asarray(v1)
            v2 = np.asarray(v2)
            
            norm1 = LA.norm(v1)
            norm2 = LA.norm(v2)
            
            v1 = v1/norm1 #[gradient_tev[0][k][j][i]/norm1, gradient_tev[1][k][j][i]/norm1, gradient_tev[2][k][j][i]/norm1]
            v2 = v2/norm2 #[gradient_vo2[0][k][j][i]/norm2, gradient_vo2[1][k][j][i]/norm2, gradient_vo2[2][k][j][i]/norm2]
            
            
            zz[k,j,i] = np.dot(v1,v2)

(3, 250, 250, 50) (3, 250, 250, 50)


/Users/sdutta/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in divide
  app.launch_new_instance()
/Users/sdutta/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in divide


In [12]:
corr_arr = vtk.vtkDoubleArray()
corr_arr.SetName('corr')
corr_arr.SetNumberOfTuples(dims[0]*dims[1]*dims[2])

index=0
for i in range(dims[2]):
    for j in range(dims[1]):
        for k in range(dims[0]):
            corr_arr.SetTuple1(index,zz[k,j,i])
            index=index+1
            
tevfield.GetPointData().AddArray(corr_arr)         

1

In [13]:
outfile = 'corr_grad_P_Vel.vti'
writer = vtk.vtkXMLImageDataWriter()
writer.SetInputData(tevfield)
writer.SetFileName(outfile)
writer.Write() 

1